In this assignment you will work on the autoencoder. Always show how you arrived at your answer. Hand in your assignment by adding the solutions to this notebook file.

<H3>Exercise 1 (7 points)</H3>

We reuse the code you implemented in the MLP assignment. We will use the code to train an autoencoder and reconstruct face images taken from the Yale face database (http://vismod.media.mit.edu/vismod/classes/mas622-00/datasets). For illustration we test reconstruction performance on the training set.

In [1]:
# repeat your MLP code here

First we construct a dataset from the face database.

In [ ]:
import os
from PIL import Image
import scipy.stats as stat

n = len(os.listdir("yalefaces"))

maxsz = [32, 32]

X = []
i=0
for file in os.listdir("yalefaces"):
    im = Image.open(os.getcwd() + '/yalefaces/' + file)
    im.thumbnail(maxsz, Image.ANTIALIAS)
    data = np.asarray(im)
    if i==0:
        sz = data.shape
    X.append(np.ndarray.flatten(data))
    i+=1

# convert to numpy array
X = np.array(X).astype('float32')

# zscore operation
mu = np.mean(X,0)
X -= mu
sigma = np.std(X,0)
X /= sigma

# make suitable for training
X = X.transpose()

Now run the linear autoencoder in the undercomplete setting (e.g. nhidden=30).

Plot the first three original faces and their reconstructions by the autoencoder. Make sure to take the previously applied zscoring operation into account.

<H3>Exercise 2 (3 points)</H3>

Plot the rows and columns of the weight matrices *W1* and *W2* reshaped to the image width and height to check the receptive and projective field structure.